# get_frequently_mutated Final 

In [1]:
import cptac
import pandas as pd
import numpy as np

In [13]:
def get_frequently_mutated(cancer_object, cutoff = 0.1):  
    """
    Takes a cancer object and find the frequently 
    mutated genes (in the tumor samples) compared to the cutoff.
    Parameters:
    cancer_object (object): cancer type from cptac module 
    cutoff (float): used as a comparison to determine the 
                    status of gene mutation frequency
    Returns:
    freq_mutated_df (pd.DataFrame): DataFrame of frequently 
        mutated genes passing the cutoff. Columns contain the 
        fractions of total unique mutations,missense type 
        mutations, and truncation type mutations per gene.
    
    The Missense_Mut column includes: 
        In_Frame_Del, In_Frame_Ins, Missense_Mutation
   
    The Truncation_Mut column includes: 
        Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, 
        Nonsense_Mutation, Nonstop_Mutation
        
    These columns count multiple mutations of one gene in the 
    same sample, so fractions in the last two columns may 
    exceed the Unique_Samples_Mut column which only counts if 
    the gene was mutated once per sample."""    
    
    # Get total tumors/patients
    omics_and_mutations = cancer_object.join_omics_to_mutations(
        mutations_genes = 'TP53', omics_df_name = 'proteomics', omics_genes = 'TP53')
    tumors = omics_and_mutations.loc[omics_and_mutations['Sample_Status'] == 'Tumor'] 
    total_tumor_samples = len(tumors)
    
    # Get mutations data frame
    somatic_mutations = cancer_object.get_mutations() 
    

    # Drop silent mutations for Ovarian and RenalCcrcc dataset 
    if 'Silent' in somatic_mutations['Mutation'].unique():
        origin_df = somatic_mutations.loc[somatic_mutations['Mutation'] != 'Silent'].reset_index()
    else:
        origin_df = somatic_mutations.reset_index() #prepare to count unique samples
        
    # Create two categories in Mutation column - 'M': Missense, 'T': Truncation
    if cancer_object.get_cancer_type() == 'colon':
        missense_truncation_groups = {'frameshift substitution': 'T', 
            'frameshift deletion': 'T', 'frameshift insertion': 'T', 
            'stopgain': 'T', 'stoploss': 'T', 'nonsynonymous SNV': 'M',
            'nonframeshift insertion': 'M','nonframeshift deletion': 'M', 
            'nonframeshift substitution': 'M'}
    else: 
        missense_truncation_groups = {'In_Frame_Del': 'M', 'In_Frame_Ins': 'M',
            'Missense_Mutation': 'M', 'Frame_Shift_Del': 'T','Nonsense_Mutation': 'T', 
            'Splice_Site': 'T', 'Frame_Shift_Ins': 'T','Nonstop_Mutation':'T'}
    mutations_replaced_M_T = origin_df.replace(missense_truncation_groups)
    # Check that all mutation names are catagorized
    if len(mutations_replaced_M_T['Mutation'].unique()) != 2:
        print('Warning: New mutation name not classified. Counts will be affected.')
    
    # Find frequently mutated genes (total fraction > cutoff)
    # Step 1 - group by gene and count unique samples
    count_mutations = origin_df.groupby(['Gene']).nunique()
    # Step 2 - format
    count_mutations = count_mutations.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
    count_mutations = count_mutations.drop(['Gene', 'Mutation','Location'], axis = 1)
    # Step 3 - filter using the cutoff and create fraction
    fraction_mutated = count_mutations.apply(lambda x: x / total_tumor_samples)
    fraction_greater_than_cutoff = fraction_mutated.where(lambda x: x > cutoff) #na used when false
    filtered_gene_df = fraction_greater_than_cutoff.dropna()
    
    # Create and join Missense column (following similar steps as seen above)
    miss = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'M']
    count_miss = miss.groupby(['Gene']).nunique()
    missense_df = count_miss.rename(columns={"Sample_ID": "Missense_Mut"})
    missense_df = missense_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)
    fraction_missense = missense_df.apply(lambda x: x / total_tumor_samples)
    freq_mutated_df = filtered_gene_df.join(fraction_missense, how='left').fillna(0)
    
    # Create and join Truncation column (following similar steps as seen above)
    trunc = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'T']
    count_trunc = trunc.groupby(['Gene']).nunique()
    truncation_df = count_trunc.rename(columns={"Sample_ID": "Truncation_Mut"})
    truncation_df = truncation_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)
    fraction_truncation = truncation_df.apply(lambda x: x / total_tumor_samples)
    freq_mutated_df = freq_mutated_df.join(fraction_truncation, how='left').fillna(0)
    freq_mutated_df = freq_mutated_df.reset_index() #move genes to their own column

    return freq_mutated_df

In [49]:
#en = cptac.Endometrial()
#co = cptac.Colon()
ov = cptac.Ovarian()
#rc = cptac.RenalCcrcc()

In [59]:
df = get_frequently_mutated(ov, .05)
#df.loc[df['Unique_Samples_Mut'] <= .07]
df

,Gene,Unique_Samples_Mut,Missense_Mut,Truncation_Mut
0,ACAN,0.060241,0.048193,0.012048
1,ADAMTS2,0.084337,0.060241,0.024096
2,AHNAK,0.072289,0.072289,0.024096
3,AHNAK2,0.084337,0.084337,0.000000
4,ANKS6,0.060241,0.048193,0.012048
5,APOB,0.072289,0.072289,0.000000
6,ATRX,0.072289,0.060241,0.024096
7,BIRC6,0.120482,0.108434,0.024096
8,BRCA1,0.060241,0.012048,0.048193
9,CACNA1C,0.060241,0.048193,0.012048


In [60]:
#test results
m = ov.get_mutations() #CHANGE CANCER OBJECT

#m = m.loc[m['Mutation'] != 'Silent']
gene = 'CR1'
g = m.loc[m['Gene'] == gene]
g

,Gene,Mutation,Location
Sample_ID,,,
S054,CR1,Silent,p.L1712L
S054,CR1,Silent,p.G1713G
S054,CR1,Missense_Mutation,p.F2240S
S055,CR1,Splice_Site,p.X355_splice
S062,CR1,Missense_Mutation,p.T2000N
S080,CR1,Nonsense_Mutation,p.E2456*
S096,CR1,Missense_Mutation,p.H1490N


In [56]:

print(len(g.index.unique()))

r = g.groupby('Mutation')

r.groups

6


{'Missense_Mutation': Index(['S002', 'S053', 'S078', 'S080', 'S080', 'S080', 'S080', 'S080', 'S080',
        'S084'],
       dtype='object', name='Sample_ID'),
 'Nonsense_Mutation': Index(['S080'], dtype='object', name='Sample_ID'),
 'Silent': Index(['S049', 'S080'], dtype='object', name='Sample_ID')}

In [27]:
gene = 'KMT2B'
g = m.loc[m['Gene'] == gene]
g = g.reset_index()
g[['Mutation', 'Sample_ID']]

,Mutation,Sample_ID
0,Nonsense_Mutation,S012
1,Missense_Mutation,S017
2,Missense_Mutation,S022
3,Missense_Mutation,S022
4,Missense_Mutation,S022
5,Missense_Mutation,S022
6,Nonsense_Mutation,S022
7,Missense_Mutation,S022
8,Missense_Mutation,S022
9,Nonsense_Mutation,S022
